# Match de CVs com vagas de emprego

In [1]:
!pip install pdfplumber
import pdfplumber
import nltk

     |████████████████████████████████| 45 kB 3.3 MB/s 
     |████████████████████████████████| 5.6 MB 22.1 MB/s 
     |████████████████████████████████| 138 kB 70.9 MB/s 
     |████████████████████████████████| 1.9 MB 45.8 MB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.5.28-py3-none-any.whl size=32240 sha256=bef14c79f522bbcc67533a523bceafff0d8872bef7848ad2e95a5f5820c31c8d
  Stored in directory: /root/.cache/pip/wheels/f2/b1/a0/c0a77b756d580f53b3806ae0e0b3ec945a8d05fca1d6e10cc1
Successfully built pdfplumber


In [ ]:
cvPdf = pdfplumber.open('/content/Juliana.pdf')
primeira_pagina = cvPdf.pages[0]
texto=primeira_pagina.extract_text()
texto

In [ ]:
cvPdf.metadata

In [ ]:
cvPdf.hyperlinks

In [ ]:
print(texto)

# Pré Processamento

In [ ]:
nltk.download('punkt')
lista_palavras=nltk.tokenize.word_tokenize(texto)
lista_palavras

In [ ]:
lista_palavras=[palavra.lower() for palavra in lista_palavras]
lista_palavras

In [ ]:
pontuacao=['(', ')', ',', ':', ';', '-', '_', '%', '@', '[', ']', '{', '}' ]

In [ ]:
nltk.download('stopwords')
stop_words=nltk.corpus.stopwords.words('portuguese')
stop_words

In [ ]:
keywords=[palavra for palavra in lista_palavras if not palavra in stop_words and not palavra in pontuacao]
keywords

In [ ]:
len(keywords)

In [ ]:
textoCv=" ".join(p for p in keywords)
textoCv

# Word Cloud - Nuvem de Palavras

In [ ]:
from wordcloud import WordCloud

import matplotlib.pyplot as plt

In [ ]:
wordcloud=WordCloud(
    background_color='#f054c9',
    width=1280 ,
    height=720 ,
    max_font_size=150,
    colormap= 'Blues'
).generate(textoCv)

fig,ax = plt.subplots(figsize=(16,9))
ax.imshow(wordcloud)
ax.set_axis_off()
plt.imshow(wordcloud)
wordcloud.to_file('MYwordcloud.png')
plt.show()

In [ ]:
from wordcloud import WordCloud

import matplotlib.pyplot as plt

def NuvemDePalavras(cv, salvar = True):
  cvPdf = pdfplumber.open(cv)
  primeira_pagina=cvPdf.pages[0]
  texto=primeira_pagina.extract_text()


  nltk.download('punkt')
  lista_palavras=nltk.tokenize.word_tokenize(texto)
  lista_palavras=[palavra.lower() for palavra in lista_palavras]

  pontuacao=['(', ')', ',', ':', ';', '-', '_', '%', '@', '[', ']', '{', '}' ]
  nltk.download('stopwords')
  stop_words=nltk.corpus.stopwords.words('portuguese')

  keywords=[palavra for palavra in lista_palavras if not palavra in stop_words and not palavra in pontuacao]
  textoCv=" ".join(p for p in keywords)

  wordcloud=WordCloud(
    background_color='#f054c9',
    width=1280 ,
    height=720 ,
    max_font_size=150,
    colormap= 'Blues').generate(textoCv)

  fig,ax = plt.subplots(figsize=(16,9))
  ax.imshow(wordcloud)
  ax.set_axis_off()
  plt.imshow(wordcloud)
  wordcloud.to_file('MYwordcloud.png')
  plt.show()

  if salvar:
    wordcloud.to_file('CurriculumVwordcloud.png')

  plt.show()

In [ ]:
NuvemDePalavras('/content/Juliana.pdf')

#Vagas

In [ ]:
import pandas as pd

In [ ]:
vagas=pd.read_excel('/content/vagas.xlsx', sheet_name=None)

In [ ]:
n_vagas = len(vagas.keys())
n_vagas
nome_vagas=list(vagas.keys())
nome_vagas

In [ ]:
n_vagas

In [ ]:
vaga=[vagas[nome_vagas[i]] for i in range(n_vagas)]

In [ ]:
vaga[0]

In [ ]:
vaga1=vaga[0]

In [ ]:
palavras_chaves=list(vaga1['palavras-chave'])
palavras_chaves

In [ ]:
import numpy as np

In [ ]:
limite = 5
  
pesos = list(vaga1['pesos'])
palavras_chaves = list(vaga1['palavras-chave'])
pmax = np.sum(np.array(pesos) * limite)
print(pmax)

In [ ]:
cont = [textoCv.count(pc) for pc in palavras_chaves]
cont

In [ ]:
def aux(x):
    return x if x <= limite else limite

cont = [aux(i) for i in cont]
cont

In [ ]:
score = ((np.array(cont) * pesos).sum()/pmax).round(4)
score

In [ ]:
pontuacao=['(', ')', ',', ':', ';', '-', '_', '%', '@', '[', ']', '{', '}' ]
stop_words = nltk.corpus.stopwords.words('portuguese')

def MatchCV(cv, vaga, limite = 5):
    '''
    cv: caminho de um arquivo PDF
    vaga: dataset de palavras-chave e pesos
    '''

    cvPdf = pdfplumber.open(cv)
    primeira_pagina = cvPdf.pages[0]  #lê apenas a primeira página
    texto = primeira_pagina.extract_text()

    lista_de_palavras = nltk.tokenize.word_tokenize(texto)  # transforma o texto cru em uma lista de termos
    lista_de_palavras = [palavra.lower() for palavra in lista_de_palavras]  # deixando tudo minusculo

    keywords = [palavra for palavra in lista_de_palavras if not palavra in stop_words and not palavra in pontuacao]  # tira as pontuacoes e stopwords 
    textoCv = " ".join(s for s in keywords)  # junta tudo em um texto só novamente. 

    pesos = list(vaga['pesos'])
    palavras_chaves = list(vaga['palavras-chave'])

    cont = [textoCv.count(pc) for pc in palavras_chaves]  # conta quantas vezes cada termo da vaga aparece no texto do cv

    def aux(x, limite):
        return x if x <= limite else limite

    cont = [aux(i, limite) for i in cont]   # coloca o limite na contagem de palavras

    pmax = np.sum(np.array(pesos) * limite) 

    score = ((np.array(cont) * pesos).sum()/pmax).round(4)

    return score

In [ ]:
MatchCV('/content/Juliana.pdf', vaga[2])

In [ ]:
lista_de_vagas = vaga

lista_de_cvs = ['/content/Agner.pdf',
                '/content/Juliana.pdf',
                '/content/Nayara.pdf',
                '/content/Victor.pdf',
                '/content/Rafael.pdf',
                '/content/Sabrina.pdf']

In [ ]:
lista_de_vagas[0]

In [ ]:
pessoas = [[MatchCV(cv, vaga) for vaga in lista_de_vagas] for cv in lista_de_cvs]
pessoas

In [ ]:
nomes = [cv.split('/')[-1].split('.')[0] for cv in lista_de_cvs]

In [ ]:
matchs = pd.DataFrame(pessoas, columns = nome_vagas, index = nomes)

In [ ]:
matchs.sort_values(by = 'eng_dados', ascending = False)